In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime
#now = datetime.datetime.now()

#train = pd.read_csv('/Users/xiechenwei/Downloads/train_wity_lat_lon.csv', parse_dates=['timestamp'])
#test = pd.read_csv('/Users/xiechenwei/Downloads/test_wity_lat_lon.csv', parse_dates=['timestamp'])
train = pd.read_csv('/Users/xiechenwei/Downloads/train.csv', parse_dates=['timestamp'])
test = pd.read_csv('/Users/xiechenwei/Downloads/test.csv', parse_dates=['timestamp'])
macro = pd.read_csv('/Users/xiechenwei/Downloads/macro.csv', parse_dates=['timestamp'])
df_fixup = pd.read_excel('/Users/xiechenwei/Downloads/BAD_ADDRESS_FIX.xlsx')
id_test = test.id
train.head(1)

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000


In [2]:

#df_all = pd.concat([train.drop(['price_doc'], axis=1), test])

#fix the data
#-----------------------------------
for i in (list(df_fixup['id'])):
    (train.loc[train['id'] == i, df_fixup.columns]) = df_fixup.loc[df_fixup['id']== i].values


In [3]:
#merge the macro data
#ignore the macro
#--------------------------------------------------------
#train = pd.merge(train, macro, on = 'timestamp', how = 'left')

train.head(1)

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000


In [4]:
#clean data
bad_index = train[train.life_sq > train.full_sq].index
train.ix[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]
bad_index = test[test.life_sq > test.full_sq].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.ix[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.ix[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
test.ix[bad_index, "full_sq"] = np.NaN
kitch_is_build_year = [13117]
train.ix[kitch_is_build_year, "build_year"] = train.ix[kitch_is_build_year, "kitch_sq"]
bad_index = train[train.kitch_sq >= train.life_sq].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[test.kitch_sq >= test.life_sq].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
train.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
test.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index
train.ix[bad_index, "full_sq"] = np.NaN
bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
test.ix[bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > 300].index
train.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test[test.life_sq > 200].index
test.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)
bad_index = train[train.build_year < 1500].index
good_index = train[train.build_year >= 1500].index

# year #
train['year'] = train['timestamp'].dt.year
test['year'] = test['timestamp'].dt.year

train.ix[bad_index, "build_year"] = np.NaN
train['age_build'] = np.NaN
test['age_build'] = np.NaN
train.ix[good_index, "age_build"] = train['year'] - train['build_year']
good_index = test[train.build_year >= 1500].index
test.ix[good_index, "age_build"] = test['year'] - test['build_year']
bad_index = test[test.build_year < 1500].index
test.ix[bad_index, "build_year"] = np.NaN
bad_index = train[train.num_room == 0].index 
train.ix[bad_index, "num_room"] = np.NaN
bad_index = test[test.num_room == 0].index 
test.ix[bad_index, "num_room"] = np.NaN
bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.ix[bad_index, "num_room"] = np.NaN
bad_index = [3174, 7313]
test.ix[bad_index, "num_room"] = np.NaN
bad_index = train[(train.floor == 0).values * (train.max_floor == 0).values].index
train.ix[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train[train.floor == 0].index
train.ix[bad_index, "floor"] = np.NaN
bad_index = train[train.max_floor == 0].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.max_floor == 0].index
test.ix[bad_index, "max_floor"] = np.NaN
bad_index = train[train.floor > train.max_floor].index
train.ix[bad_index, "max_floor"] = np.NaN
bad_index = test[test.floor > test.max_floor].index
test.ix[bad_index, "max_floor"] = np.NaN
train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.ix[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.ix[bad_index, "state"] = np.NaN
test.state.value_counts()

# brings error down a lot by removing extreme price per sqm
train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]

# Add month-year
month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.timestamp.dt.month + test.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.timestamp.dt.weekofyear + test.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek





/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [5]:
train['floor_from_top'] =  train['max_floor'] - train['floor']
test['floor_from_top'] = test['max_floor'] - test['floor']


train['season'] = (train['month'].astype(int) / 4).astype(int) + 1
test['season'] = (test['month'].astype(int) / 4).astype(int) + 1

train['edu_ratio'] = train['preschool_education_centers_raion'] / train['school_education_centers_raion']
test['edu_ratio'] = test['preschool_education_centers_raion'] / test['school_education_centers_raion']

train['school_ratio'] = train['children_school'] / train['school_quota'].astype(float)
test['school_ratio'] = test['children_school'] / test['school_quota'].astype(float)
#好像没有一点用
#---------------------------------------
#train['work_ratio'] = train['work_all'] / train['full_all'].astype(float)
#test['work_ratio'] = test['work_all'] / test['full_all'].astype(float)

#train['male_ratio'] = train['male_f'] / train['full_all'].astype(float)
#test['male_ratio'] = test['male_f'] / test['full_all'].astype(float)

#train['young_ratio'] = train['young_all'] / train['full_all'].astype(float)
#test['young_ratio'] = test['young_all'] / test['full_all'].astype(float)

#train['ekder_ratio'] = train['ekder_all'] / train['full_all'].astype(float)
#test['ekder_ratio'] = test['ekder_all'] / test['full_all'].astype(float)


#train['work_male_ratio'] = train['work_male'] / train['work_all'].astype(float)
#test['work_male_ratio'] = test['work_male'] / test['work_all'].astype(float)
#------------------------------------------


In [6]:
# Other feature engineering
train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)


test['rel_floor'] = test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = test['kitch_sq'] / test['full_sq'].astype(float)

train.apartment_name=train.sub_area + train['metro_km_avto'].astype(str)
test.apartment_name=test.sub_area + train['metro_km_avto'].astype(str)

train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)



In [7]:
y_train = train["price_doc"]
num_train = train.shape[0]

In [8]:
df_all = pd.concat([train.drop(['price_doc'], axis=1), test])
df_all.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,week_year_cnt,month,dow,floor_from_top,season,edu_ratio,school_ratio,rel_floor,rel_kitch_sq,room_size
0,1,2011-08-20,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,1,8,5,NaN,3,1.000000,0.931676,NaN,NaN,NaN
1,2,2011-08-23,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,2,8,1,NaN,3,0.625000,1.244028,NaN,NaN,NaN
2,3,2011-08-27,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,2,8,5,NaN,3,0.571429,1.112366,NaN,NaN,NaN
3,4,2011-09-01,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,1,9,3,NaN,3,0.900000,0.801149,NaN,NaN,NaN
4,5,2011-09-05,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,5,9,0,NaN,3,0.777778,0.868468,NaN,NaN,NaN


In [9]:
#clean some feature
df_all.loc[df_all['full_sq'] < 10, ['full_sq']] = train['full_sq'].mean()
df_all.loc[df_all['life_sq'] < 10, ['life_sq']] = train['life_sq'].mean()
df_all.loc[df_all['kitch_sq'] < 2, ['kitch_sq']] = train['kitch_sq'].mean()
df_all.loc[df_all['max_floor'] < df_all['floor'], ['max_floor']] = df_all.loc[df_all['max_floor'] < df_all['floor'], ['floor']] + 1
df_all['floor_from_top'] = df_all.apply(lambda x: x['max_floor'] - x['floor'], axis=1)

In [10]:
df_all.drop(["id", "timestamp", "year"], axis=1, inplace = True)
#-------------------------------
for c in df_all.columns:
    if df_all[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_all[c].values)) 
        df_all[c] = lbl.transform(list(df_all[c].values))
        #x_train.drop(c,axis=1,inplace=True)
#--------------------------------
print (df_all.shape)

(38058, 301)


In [11]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=1035, num_rounds=400):
    xgb_params = {
        'eta': 0.05,
        'max_depth': 5,
        'subsample': 0.7,
        'colsample_bytree': 0.7,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'silent': 1,
        'seed': seed_val,
        #'lambda': 5,
        #'gamma': 1.0,
        'colsample_bylevel': 0.7
    }
    num_rounds = num_rounds

    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [(xgtrain, 'train'), (xgtest, 'test')]
        model = xgb.train(xgb_params, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval = 50)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(xgb_params, xgtrain, num_rounds, verbose_eval = 50)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [12]:
x_train = df_all[0:num_train].copy().values
x_test = df_all[num_train:].copy().values
#-----Imputation transformer for completing missing values.---#
imputer = preprocessing.Imputer()
x_train = imputer.fit_transform(x_train)
x_test = imputer.transform(x_test)
#---------------dont know whether effect is good--------------#

In [13]:
print(num_train)

30396


In [14]:
y_pred, model = runXGB(x_train, y_train, x_test, num_rounds=500)
output = pd.DataFrame({'id': id_test, 'price_doc': y_pred})
output.to_csv('/Users/xiechenwei/Desktop/数据挖掘结果/submission1.csv', index=False)
output.head()

,id,price_doc
0,30474,5057481.0
1,30475,8118254.0
2,30476,5109860.5
3,30477,5776085.5
4,30478,4839074.5


In [15]:
#把这次上一次结果当作特征输入 6/24 submission1
xg_test = xgb.DMatrix(x_train)
pre_train = model.predict(xg_test)
pre = pd.DataFrame({'price_doc': y_pred})
pre.to_csv('/Users/xiechenwei/Desktop/数据挖掘结果/32092_train_feat.csv', index=False)